In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model

## Read in data and organize it so that each column is a county's case counts

In [ ]:
data = pd.read_csv("/home/nathan/Documents/Fun/COVID.csv")

In [ ]:
fips = data['fips'].drop_duplicates()
col_names = ["date"]
col_names += fips.values.tolist()

In [ ]:
merged_counties = data.loc[data['fips'] == 53061][['date','cases']]
for i in range(1,len(fips)):
    merged_counties = merged_counties.merge(data.loc[data['fips'] == fips.iloc[i]][['date','cases']],how = 'outer',on = 'date')

In [ ]:
merged_counties = merged_counties.set_axis(col_names, axis=1, inplace=False)

In [ ]:
fips_keep = merged_counties.isna().sum() < 50
fips_keep = fips_keep.drop('date')

In [ ]:
merged_counties_zero = merged_counties.fillna(0)

## Perpare data to be correct shape and formatfor model

In [ ]:
inputs = merged_counties_zero.iloc[0:75]
inputs.shape

In [ ]:
outputs = merged_counties_zero.iloc[5:]
outputs.shape

In [ ]:
outputs = np.delete(outputs.to_numpy(),0,axis = 1)

In [ ]:
inputs = np.delete(inputs.to_numpy(),0,axis = 1)

In [ ]:
X = inputs[:,fips_keep]
p = X.shape[1]

In [ ]:
X_ = np.kron(np.identity(p),X)
Y_ = outputs[:,fips_keep].flatten('F')

## Run model and save network

In [ ]:
reg = linear_model.ElasticNet(alpha = 500, random_state=0)
coefs = reg.fit(X_, Y_).coef_
network = pd.DataFrame(np.reshape(coefs,(p,p),'F'))
network.set_axis(list(fips_keep.index[fips_keep]),axis = 1)
network.set_axis(list(fips_keep.index[fips_keep]),axis = 0)
network.to_csv("/home/nathan/Documents/Fun/COVID_network.csv")